In [ ]:
import random
import os
from pathlib import Path

import numpy as np
import PIL
import tqdm
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib

import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import dnnlib
import dnnlib.tflib as tflib
import pretrained_networks

from sklearn.svm import SVC
import pandas as pd
from scipy.spatial.distance import euclidean, cosine

In [ ]:
# Генерируем картинку и латентный вектор
network_pkl = 'latest.pkl'
print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)


def generate_img(Gs, W):
    X = Gs.components.synthesis.run(W, randomize_noise=False, minibatch_size=1, print_progress=False,
                                    output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
    return PIL.Image.fromarray(X[0])

In [ ]:
def linear_interpolate(latent_code,
                       boundary,
                       start_distance=-3.0,
                       end_distance=3.0,
                       steps=10):
    """Manipulates the given latent code with respect to a particular boundary.
        Basically, this function takes a latent code and a boundary as inputs, and
      outputs a collection of manipulated latent codes.
      NOTE: Distance is sign sensitive.
      Args:
        latent_code: The input latent code for manipulation.
        boundary: The semantic boundary as reference.
        start_distance: The distance to the boundary where the manipulation starts.
          (default: -3.0)
        end_distance: The distance to the boundary where the manipulation ends.
          (default: 3.0)
        steps: Number of steps to move the latent code from start position to end
      position. (default: 10)
      """
    assert (latent_code.shape[0] == 1 and boundary.shape[0] == 1 and
          len(boundary.shape) == 2 and
          boundary.shape[1] == latent_code.shape[-1])

    linspace = np.linspace(start_distance, end_distance, steps)
    if len(latent_code.shape) == 2:
        linspace = linspace - latent_code.dot(boundary.T) # linspace - латентный код z * вектор нормали n^T
        linspace = linspace.reshape(-1, 1).astype(np.float32)
        return latent_code + linspace * boundary # эта формула из статьи, где single attribute manipulation
                                                                    # z_edit = z + alpha * n
        

In [ ]:
latents_base_path = Path('/media/svakhreev/fast/generated_images/latents/')
base_path = Path('/media/svakhreev/fast/generated_images/crops/body')
all_filenames = [path for path in base_path.iterdir() if path.suffix == '.npy']

image_base_path = base_path / f'030131_1' # опорная картинка
image = cv2.imread(f'{image_base_path}.png')
feat_1 = np.load(f'{image_base_path}.npy')[0] # латетный вектор опорной картинки

PIL.Image.open(f'{image_base_path}.png')

In [ ]:
# списки признаков латентного пространства

dist_thresh = 22. # пороговое расстояние
distances = []
for filename in tqdm.tqdm(all_filenames):
    if filename.suffix != '.npy':
        continue
    feat_2 = np.load(str(filename))[0]
    distances.append(euclidean(feat_1, feat_2))
    
sorted_dist_idxes = np.argsort(distances)
top_k_nearest = (np.array(distances) <= dist_thresh).sum()
min_idxes = sorted_dist_idxes[:top_k_nearest] 
max_idxes = sorted_dist_idxes[-5000:]

min_features = [np.load(str(latents_base_path / all_filenames[idx].name))[0]
                for idx in min_idxes]
max_features = [np.load(str(latents_base_path / all_filenames[idx].name))[0]
                for idx in max_idxes]
print(top_k_nearest)


In [ ]:
X = pd.DataFrame(min_features + max_features)
y = pd.DataFrame([[0 if idx < len(min_features) else 1] for idx in range(len(X))])

In [ ]:
svc = SVC(kernel='linear', class_weight='balanced')
svc.fit(X, y) 
print(svc.score(X, y))

boundary = svc.coef_.reshape(1, 512).astype(np.float32) # разделяющая граница
boundary /= np.linalg.norm(boundary)

In [ ]:
# генерация произвольной картинки с помощью обученного stylegan2

truncation = 0.7
Z = np.random.randn(1, Gs.input_shape[1])
W = Gs.components.mapping.run(Z, None, minibatch_size=1)

dlatent_avg = Gs.get_var('dlatent_avg')
W = (W[np.newaxis] - dlatent_avg) * np.reshape([truncation, -truncation], [-1, 1, 1, 1]) + dlatent_avg
W = np.append(W[0], W[1], axis=0)
W = W[:, :512]
W = W.reshape((2, 16, 512))[[0]]

generate_img(Gs, W) # эту картинку будем изменять

In [ ]:
new_latents = linear_interpolate(latent_code=W, boundary=boundary, 
                                 start_distance=0, end_distance=-30,
                                 steps=150)

In [ ]:
writer = cv2.VideoWriter('./out.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 25,
                          (512, 1024))
for feat in tqdm.tqdm(new_latents):
    image = generate_img(Gs, feat[None, ])
    writer.write(np.array(image)[..., ::-1])
writer.release()